In [54]:
library(data.table)
library(TSrepr)
library(TSdist)
library(dtw)
library(TunePareto)
library(dplyr)

# Summary

In this homework, 60 different parameter combination will be tried to get hyperparameter tuning. To get this combination, 5 different representation types(raw + 2 different difference taken + 2 different PAA), 4 different distance calculations, and 3 different k values will be used. Difference will be taken by shift function and 1, 2 will be used to get difference values. For PAA, 2 different segment length will be used by considering proper values for timeseries. In terms of distance, Euclidean, Dynamic Time Wrapping, Longest Common Subsequence, and Edit Distance with Real Penalties will be used. Lastly, k values are selected as 1, 3, 5 for k-nearest neighbor model. 

Performance of the models will be inspected by using 10-fold cross and 5 repeated validation technique. Moreover, performance of the models will be controlled by having the same test indices to have identical conditions. Lastly, obtained 60 performance will be controlled by a dataframe including average accuracy and standard deviation in accuracy. By considering this dataframe, the best parameter combination will be controlled and this parameters will be used to get accuracy performance in the test dataset. Last comments will be held at the end of the notebooks.

### Parameter Selections

- Representation Type(5): Raw, 1 Difference Taken, 2 Difference Taken, Piecewise Aggregate Approximation(PAA) with 8 segment length, Piecewise Aggregate Approximation(PAA) with 16 segment length
- Distance Calculation(4): Euclidean, Dynamic Time Warping with default parameters, Longest Common Subsequence(LCSS) with epsilon 0.05, Edit Distance with Real Penalties(ERP) with epsilon 0.5
- k values of nearest neighbor model(3): 1, 3, 5

# Context
(In order to get specified techniques rapidly, a context part is added to Notebook.)

1. [Data Preparation](#1)
1. [Classify Function](#2)
1. [Representations](#3)
    1. Raw Data
    1. [Difference Function](#4)
    1. [PAA Function](#5)
1. [Distances](#6)
    1. [Raw Data](#7)
    1. [Difference Data](#8)
    1. [PAA Data](#9)
1. [Main Model](#10)
1. [Result of Models](#11)
1. [Test Performance](#12)
1. [Comments](#13)

<a id="1"></a>
# Data Preparation

In [2]:
dataset_path="D:/Datasets/Univariate2018_arff/Univariate_arff/"

In [3]:
distance_path="C:/Users/bahad/GitHub/IE48B/Homework3/Distances/"

In [4]:
first_dataset="Beef"
second_dataset="BirdChicken"
third_dataset="BMW"
fourth_dataset="Coffee"
fifth_dataset="Wine"

## Loading Dataset

In [5]:
traindata=as.matrix(fread(sprintf('%s%s/%s_TRAIN.txt',dataset_path, second_dataset,second_dataset)))

In [6]:
head(traindata)

V1,V2,V3,V4,V5,V6,V7,V8,V9,V10,...,V504,V505,V506,V507,V508,V509,V510,V511,V512,V513
1,1.7613808,1.7294248,1.6788066,1.6146098,1.5291964,1.4652777,1.4097854,1.3462449,1.2914795,...,1.44265740,1.463438800,1.52507210,1.57726710,1.6092094,1.6414753,1.6949783,1.7279482,1.7314824,1.7567164
1,0.3915731,0.3694528,0.3548357,0.3337060,0.3208962,0.3007925,0.2528962,0.2055984,0.1868509,...,-0.03092597,-0.009834623,0.04031407,0.05580091,0.1073028,0.1303724,0.1882483,0.2517759,0.3153889,0.3502306
1,0.5248411,0.5008023,0.4661912,0.4441019,0.4124850,0.3924422,0.3732443,0.3099562,0.2560531,...,0.04026058,0.104056700,0.15591906,0.19375546,0.2575487,0.2954536,0.3456947,0.3838166,0.4476355,0.4858116
1,1.6561027,1.6190680,1.5642555,1.5282153,1.4668354,1.4021028,1.3593206,1.2957290,1.2328644,...,1.77110620,1.748840300,1.70784710,1.70701040,1.6864602,1.6666652,1.6874689,1.6884775,1.6898214,1.6725704
1,1.6256061,1.6350423,1.6556782,1.6668675,1.6786290,1.6909626,1.7038633,1.7313345,1.7458998,...,1.18152600,1.233014300,1.24070270,1.30033710,1.3516590,1.4029964,1.4116524,1.4629014,1.5232277,1.5744039
1,1.7790179,1.7313825,1.6839585,1.5897730,1.5430287,1.4965186,1.4634212,1.4308150,1.3987001,...,1.82314030,1.799433900,1.82685900,1.85479820,1.8052034,1.7557743,1.7848103,1.7656967,1.7960351,1.7478856


In [7]:
str(traindata)

 num [1:20, 1:513] 1 1 1 1 1 1 1 1 1 1 ...
 - attr(*, "dimnames")=List of 2
  ..$ : NULL
  ..$ : chr [1:513] "V1" "V2" "V3" "V4" ...


## Class Information

In [8]:
trainclass=traindata[,1] 

## Time Series

In [9]:
traindata=traindata[,2:ncol(traindata)]

In [10]:
traindata

V2,V3,V4,V5,V6,V7,V8,V9,V10,V11,...,V504,V505,V506,V507,V508,V509,V510,V511,V512,V513
1.7613808,1.7294248,1.6788066,1.6146098,1.5291964,1.4652777,1.40978540,1.34624490,1.2914795,1.2581905,...,1.44265740,1.463438800,1.52507210,1.57726710,1.60920940,1.64147530,1.694978300,1.72794820,1.73148240,1.75671640
0.3915731,0.3694528,0.3548357,0.3337060,0.3208962,0.3007925,0.25289618,0.20559839,0.1868509,0.1783306,...,-0.03092597,-0.009834623,0.04031407,0.05580091,0.10730282,0.13037240,0.188248290,0.25177587,0.31538894,0.35023064
0.5248411,0.5008023,0.4661912,0.4441019,0.4124850,0.3924422,0.37324430,0.30995625,0.2560531,0.1927393,...,0.04026058,0.104056700,0.15591906,0.19375546,0.25754872,0.29545362,0.345694750,0.38381657,0.44763553,0.48581159
1.6561027,1.6190680,1.5642555,1.5282153,1.4668354,1.4021028,1.35932060,1.29572900,1.2328644,1.1913725,...,1.77110620,1.748840300,1.70784710,1.70701040,1.68646020,1.66666520,1.687468900,1.68847750,1.68982140,1.67257040
1.6256061,1.6350423,1.6556782,1.6668675,1.6786290,1.6909626,1.70386330,1.73133450,1.7458998,1.6960657,...,1.18152600,1.233014300,1.24070270,1.30033710,1.35165900,1.40299640,1.411652400,1.46290140,1.52322770,1.57440390
1.7790179,1.7313825,1.6839585,1.5897730,1.5430287,1.4965186,1.46342120,1.43081500,1.3987001,1.3360240,...,1.82314030,1.799433900,1.82685900,1.85479820,1.80520340,1.75577430,1.784810300,1.76569670,1.79603510,1.74788560
1.7641677,1.7210175,1.6750098,1.6426975,1.6218026,1.6203705,1.58958210,1.54805720,1.5181645,1.4887772,...,1.87411370,1.865217200,1.84521940,1.84436290,1.83657960,1.83668370,1.829642200,1.82292490,1.82441640,1.79956440
1.4463001,1.4279428,1.3736423,1.3558062,1.2910904,1.2677035,1.22116830,1.17496580,1.1291108,1.0836091,...,1.48369790,1.463650300,1.46468090,1.46612090,1.46797610,1.47024060,1.472914300,1.47599750,1.47948990,1.46065850
1.5684740,1.5303560,1.4927556,1.4689787,1.4027595,1.3555841,1.30875980,1.26230170,1.2162218,1.1705292,...,1.56878870,1.568788700,1.56921240,1.57005370,1.57131860,1.57300420,1.575110400,1.57763430,1.60354160,1.58394090
1.5134320,1.4679440,1.4229359,1.3784239,1.3331822,1.2879406,1.24194070,1.19643350,1.1514414,1.1522222,...,1.56186370,1.557150500,1.55290440,1.54912540,1.54582010,1.54298190,1.540614000,1.53872300,1.53730230,1.51414870


## Dataset Information

In [11]:
tlength=ncol(traindata)
n_series_train=nrow(traindata)

## Indices for Datasets

Mentioned test indices are obtained by TunePareto library to have identical conditions. nfold and ntimes parameters are selected as 10, 5 respectively.

In [12]:
set.seed(35)
nof_rep=5
n_fold=10

In [13]:
cv_indices=generateCVRuns(trainclass, ntimes =nof_rep, nfold = n_fold, 
                          leaveOneOut = FALSE, stratified = TRUE)

str(cv_indices)

List of 5
 $ Run  1:List of 10
  ..$ Fold  1 : int [1:2] 10 19
  ..$ Fold  2 : int [1:2] 6 20
  ..$ Fold  3 : int [1:2] 8 11
  ..$ Fold  4 : int [1:2] 1 13
  ..$ Fold  5 : int [1:2] 3 15
  ..$ Fold  6 : int [1:2] 7 18
  ..$ Fold  7 : int [1:2] 2 17
  ..$ Fold  8 : int [1:2] 5 12
  ..$ Fold  9 : int [1:2] 4 16
  ..$ Fold  10: int [1:2] 9 14
 $ Run  2:List of 10
  ..$ Fold  1 : int [1:2] 7 18
  ..$ Fold  2 : int [1:2] 3 19
  ..$ Fold  3 : int [1:2] 10 16
  ..$ Fold  4 : int [1:2] 6 14
  ..$ Fold  5 : int [1:2] 5 13
  ..$ Fold  6 : int [1:2] 4 15
  ..$ Fold  7 : int [1:2] 2 12
  ..$ Fold  8 : int [1:2] 1 11
  ..$ Fold  9 : int [1:2] 9 20
  ..$ Fold  10: int [1:2] 8 17
 $ Run  3:List of 10
  ..$ Fold  1 : int [1:2] 10 13
  ..$ Fold  2 : int [1:2] 1 11
  ..$ Fold  3 : int [1:2] 7 17
  ..$ Fold  4 : int [1:2] 3 19
  ..$ Fold  5 : int [1:2] 2 12
  ..$ Fold  6 : int [1:2] 6 20
  ..$ Fold  7 : int [1:2] 9 18
  ..$ Fold  8 : int [1:2] 8 16
  ..$ Fold  9 : int [1:2] 4 15
  ..$ Fold  10: int [1:2]

<a id="2"></a>
# Classify Function

Classify function is obtained from lecture notebooks. Function takes 4 different parameters, distance matrix, class information, test indeces and k parameter.

In [14]:
nn_classify_cv=function(dist_matrix,train_class,test_indices,k){
    
    test_distances_to_train=dist_matrix[test_indices,]
    test_distances_to_train=test_distances_to_train[,-test_indices]
    train_class=train_class[-test_indices]

    ordered_indices=apply(test_distances_to_train,1,order)
    if(k==1){
        nearest_class=as.numeric(train_class[as.numeric(ordered_indices[1,])])
        nearest_class=data.table(id=test_indices,nearest_class)
    } else {
        nearest_class=apply(ordered_indices[1:k,],2,function(x) {train_class[x]})
        nearest_class=data.table(id=test_indices,t(nearest_class))
    }
    
    long_nn_class=melt(nearest_class,'id')

    class_counts=long_nn_class[,.N,list(id,value)]
    class_counts[,predicted_prob:=N/k]
    wide_class_prob_predictions=dcast(class_counts,id~value,value.var='predicted_prob')
    wide_class_prob_predictions[is.na(wide_class_prob_predictions)]=0
    class_predictions=class_counts[,list(predicted=value[which.max(N)]),by=list(id)]
    
    
    return(list(prediction=class_predictions,prob_estimates=wide_class_prob_predictions))
    
}

<a id="3"></a>
# Representations

There are 3 major representations, raw dataset, difference taken dataset, and paa representation. For difference taken dataset 1 and 2 will be used in shift operator. At the beginning, an example code will be given to show obtained dataframes in difference datasets. For paa dataset 8 and 16 will be used in segment length parameter. At the beginning, an example code will be given to show obtained dataframes in paa datasets.

## Example Code for difference datasets

In [15]:
dt_ts_train=data.table(traindata)
dt_ts_train[,id:=1:.N]
long_train=melt(dt_ts_train,id.vars=c('id'))
long_train[,time:=as.numeric(gsub("\\D", "", variable))-1]
long_train=long_train[order(id,time)]
diff_long=copy(long_train)
diff_long[,diff_series:=value-shift(value,1),by=list(id)]
head(diff_long)

id,variable,value,time,diff_series
1,V2,1.761381,1,NA
1,V3,1.729425,2,-0.0319560
1,V4,1.678807,3,-0.0506182
1,V5,1.614610,4,-0.0641968
1,V6,1.529196,5,-0.0854134
1,V7,1.465278,6,-0.0639187


In [16]:
diff_train=dcast(diff_long[!is.na(diff_series)],id~time,value.var='diff_series')
diff_train=diff_train[,-c("id")]
head(diff_train)
diff_train=as.matrix(diff_train)

2,3,4,5,6,7,8,9,10,11,...,503,504,505,506,507,508,509,510,511,512
-0.03195600,-0.05061820,-0.06419680,-0.08541340,-0.06391870,-0.05549230,-0.06354050,-0.05476540,-0.03328900,-0.06289560,...,0.06104080,0.02078140,0.06163330,0.05219500,0.03194230,0.03226590,0.05350300,0.03296990,0.00353420,0.02523400
-0.02212026,-0.01461712,-0.02112967,-0.01280988,-0.02010370,-0.04789628,-0.04729779,-0.01874754,-0.00852026,-0.05545838,...,0.02068449,0.02109134,0.05014869,0.01548684,0.05150191,0.02306958,0.05787589,0.06352758,0.06361307,0.03484170
-0.02403879,-0.03461106,-0.02208931,-0.03161691,-0.02004279,-0.01919792,-0.06328805,-0.05390317,-0.06331375,-0.03630650,...,0.03787350,0.06379612,0.05186236,0.03783640,0.06379326,0.03790490,0.05024113,0.03812182,0.06381896,0.03817606
-0.03703470,-0.05481250,-0.03604020,-0.06137990,-0.06473260,-0.04278220,-0.06359160,-0.06286460,-0.04149190,-0.03981550,...,0.01679450,-0.02226590,-0.04099320,-0.00083670,-0.02055020,-0.01979500,0.02080370,0.00100860,0.00134390,-0.01725100
0.00943620,0.02063590,0.01118930,0.01176150,0.01233360,0.01290070,0.02747120,0.01456530,-0.04983410,0.01527790,...,0.05147270,0.05148830,0.00768840,0.05963440,0.05132190,0.05133740,0.00865600,0.05124900,0.06032630,0.05117620
-0.04763540,-0.04742400,-0.09418550,-0.04674430,-0.04651010,-0.03309740,-0.03260620,-0.03211490,-0.06267610,-0.03054970,...,0.02608840,-0.02370640,0.02742510,0.02793920,-0.04959480,-0.04942910,0.02903600,-0.01911360,0.03033840,-0.04814950


<a id="4"></a>
## Difference Function

In [17]:
difference_obtainer=function(traindata, diff_value){
    dt_ts_train=data.table(traindata)
    dt_ts_train[,id:=1:.N]
    long_train=melt(dt_ts_train,id.vars=c('id'))
    long_train[,time:=as.numeric(gsub("\\D", "", variable))-1]
    long_train=long_train[order(id,time)]
    diff_long=copy(long_train)
    diff_long[,diff_series:=value-shift(value,diff_value),by=list(id)]#Lag value is assigned by diff_value
    head(diff_long)
    
    diff_train=dcast(diff_long[!is.na(diff_series)],id~time,value.var='diff_series')
    diff_train=diff_train[,-c("id")]
    head(diff_train)
    diff_train=as.matrix(diff_train)
    
    return(diff_train)
}

This function will be used to get a difference dataset. "_2" string will be used to mention 2 differences taken dataset.

###  1 Difference

In [18]:
diff_train=difference_obtainer(traindata,1)

### 2 Difference

In [19]:
diff_train_2=difference_obtainer(traindata,2)

## Example Code for PAA

In [20]:
segment_length=5

In [21]:
paa_results=vector("list", max(long_train$id))

In [22]:
for(i in 1:max(long_train$id)){
    current_ts=long_train[id==i,]$value
    
    paa_rep=repr_paa(current_ts, segment_length, meanC)
    current_dt=data.table(time=1:length(long_train[id==i,]$value))
    result_dt=data.table(time=c(1:(length(paa_rep)))*segment_length, values=paa_rep)
    all_dt=merge(current_dt, result_dt, by='time',all.x=T)
    all_dt[,values:=nafill(values,'nocb')]
    paa_results[[i]]=transpose(data.table(values=all_dt$values))
    
}

In [23]:
paa_train=rbindlist(paa_results)

In [24]:
paa_train

V1,V2,V3,V4,V5,V6,V7,V8,V9,V10,...,V503,V504,V505,V506,V507,V508,V509,V510,V511,V512
1.6626837,1.6626837,1.6626837,1.6626837,1.6626837,1.35419560,1.35419560,1.35419560,1.35419560,1.35419560,...,1.43496828,1.43496828,1.43496828,1.65017566,1.65017566,1.65017566,1.65017566,1.65017566,NA,NA
0.3540928,0.3540928,0.3540928,0.3540928,0.3540928,0.22489369,0.22489369,0.22489369,0.22489369,0.22489369,...,-0.03775456,-0.03775456,-0.03775456,0.14670006,0.14670006,0.14670006,0.14670006,0.14670006,NA,NA
0.4696843,0.4696843,0.4696843,0.4696843,0.4696843,0.30488704,0.30488704,0.30488704,0.30488704,0.30488704,...,0.05110972,0.05110972,0.05110972,0.29525382,0.29525382,0.29525382,0.29525382,0.29525382,NA,NA
1.5668954,1.5668954,1.5668954,1.5668954,1.5668954,1.29627786,1.29627786,1.29627786,1.29627786,1.29627786,...,1.75573166,1.75573166,1.75573166,1.68721644,1.68721644,1.68721644,1.68721644,1.68721644,NA,NA
1.6523646,1.6523646,1.6523646,1.6523646,1.6523646,1.71362518,1.71362518,1.71362518,1.71362518,1.71362518,...,1.17131572,1.17131572,1.17131572,1.38590926,1.38590926,1.38590926,1.38590926,1.38590926,NA,NA
1.6654321,1.6654321,1.6654321,1.6654321,1.6654321,1.42509578,1.42509578,1.42509578,1.42509578,1.42509578,...,1.81891310,1.81891310,1.81891310,1.79325658,1.79325658,1.79325658,1.79325658,1.79325658,NA,NA
1.6849390,1.6849390,1.6849390,1.6849390,1.6849390,1.55299030,1.55299030,1.55299030,1.55299030,1.55299030,...,1.86570710,1.86570710,1.86570710,1.83403866,1.83403866,1.83403866,1.83403866,1.83403866,NA,NA
1.3789564,1.3789564,1.3789564,1.3789564,1.3789564,1.17531150,1.17531150,1.17531150,1.17531150,1.17531150,...,1.48844944,1.48844944,1.48844944,1.47064988,1.47064988,1.47064988,1.47064988,1.47064988,NA,NA
1.4926648,1.4926648,1.4926648,1.4926648,1.4926648,1.26267932,1.26267932,1.26267932,1.26267932,1.26267932,...,1.57776804,1.57776804,1.57776804,1.57342424,1.57342424,1.57342424,1.57342424,1.57342424,NA,NA
1.4231836,1.4231836,1.4231836,1.4231836,1.4231836,1.20599568,1.20599568,1.20599568,1.20599568,1.20599568,...,1.58028932,1.58028932,1.58028932,1.54345288,1.54345288,1.54345288,1.54345288,1.54345288,NA,NA


<a id="5"></a>
## PAA Function

In [25]:
paa_obtainer=function(traindata,segment_length){
    dt_ts_train=data.table(traindata)
    dt_ts_train[,id:=1:.N]
    long_train=melt(dt_ts_train,id.vars=c('id'))
    long_train[,time:=as.numeric(gsub("\\D", "", variable))-1]
    long_train=long_train[order(id,time)]
    
    paa_results=vector("list", max(long_train$id))
    for(i in 1:max(long_train$id)){
        current_ts=long_train[id==i,]$value

        paa_rep=repr_paa(current_ts, segment_length, meanC)
        current_dt=data.table(time=1:length(long_train[id==i,]$value))
        result_dt=data.table(time=c(1:(length(paa_rep)))*segment_length, values=paa_rep)
        all_dt=merge(current_dt, result_dt, by='time',all.x=T)
        all_dt[,values:=nafill(values,'nocb')]
        paa_results[[i]]=transpose(data.table(values=all_dt$values))

    }
    return(rbindlist(paa_results))
}

This function will be used to get a difference dataset. "_2" string will be used to mention segment lentgh determined as 10.

### Segment Length 8

In [26]:
paa_train=paa_obtainer(traindata,8)

### Segment Length 16

In [27]:
paa_train_2=paa_obtainer(traindata,16)

<a id="6"></a>
# Distances

In this part, distance datasets will be calculated to have 20 different combination (5 representation * 4 different distance calculation types). In addition, obtained distance datasets will be stored in a file to skip distance calculation.

In [28]:
large_number=10000

<a id="7"></a>
## Raw Dataset 

In [29]:
dist_euc=as.matrix(dist(traindata))
diag(dist_euc)=large_number
fwrite(dist_euc,sprintf('%s%s/%s_euc_raw_dist.csv', distance_path, second_dataset, second_dataset),col.names=F)

dist_dtw=as.matrix(dtwDist(traindata))
diag(dist_dtw)=large_number
fwrite(dist_dtw,sprintf('%s%s/%s_dtw_raw_dist.csv', distance_path, second_dataset, second_dataset),col.names=F)

dist_lcss=TSDatabaseDistances(traindata,distance='lcss',epsilon=0.05)
dist_lcss=as.matrix(dist_lcss)
diag(dist_lcss)=large_number
fwrite(dist_lcss,sprintf('%s%s/%s_lcss_raw_epsilon_005.csv', distance_path, second_dataset, second_dataset),col.names=F)  

dist_erp=TSDatabaseDistances(traindata,distance='erp',g=0.5)
dist_erp=as.matrix(dist_erp)
diag(dist_erp)=large_number
fwrite(dist_erp,sprintf('%s%s/%s_erp_raw_gap_005.csv', distance_path, second_dataset, second_dataset),col.names=F)  

x being coerced from class: matrix to data.table
x being coerced from class: matrix to data.table
x being coerced from class: matrix to data.table
x being coerced from class: matrix to data.table


<a id="8"></a>
## Difference taken Datasets

### First difference dataset when shift value is 1

In [30]:
dist_euc_diff=as.matrix(dist(diff_train))
diag(dist_euc_diff)=large_number
fwrite(dist_euc_diff,sprintf('%s%s/%s_euc_diff_dist.csv', distance_path, second_dataset, second_dataset),col.names=F)

dist_dtw_diff=as.matrix(dtwDist(diff_train))
diag(dist_dtw_diff)=large_number
fwrite(dist_dtw_diff,sprintf('%s%s/%s_dtw_diff_dist.csv', distance_path, second_dataset, second_dataset),col.names=F)

dist_lcss_diff=TSDatabaseDistances(diff_train,distance='lcss',epsilon=0.05)
dist_lcss_diff=as.matrix(dist_lcss_diff)
diag(dist_lcss_diff)=large_number
fwrite(dist_lcss_diff,sprintf('%s%s/%s_lcss_diff_epsilon_005.csv', distance_path, second_dataset, second_dataset),col.names=F)  

dist_erp_diff=TSDatabaseDistances(diff_train,distance='erp',g=0.5)
dist_erp_diff=as.matrix(dist_erp_diff)
diag(dist_erp_diff)=large_number
fwrite(dist_erp_diff,sprintf('%s%s/%s_erp_diff_gap_005.csv', distance_path, second_dataset, second_dataset),col.names=F)  

x being coerced from class: matrix to data.table
x being coerced from class: matrix to data.table
x being coerced from class: matrix to data.table
x being coerced from class: matrix to data.table


### Second difference dataset when shift value is 2

In [31]:
dist_euc_diff_2=as.matrix(dist(diff_train_2))
diag(dist_euc_diff_2)=large_number
fwrite(dist_euc_diff_2,sprintf('%s%s/%s_euc_diff2_dist.csv', distance_path, second_dataset, second_dataset),col.names=F)

dist_dtw_diff_2=as.matrix(dtwDist(diff_train_2))
diag(dist_dtw_diff_2)=large_number
fwrite(dist_dtw_diff_2,sprintf('%s%s/%s_dtw_diff2_dist.csv', distance_path, second_dataset, second_dataset),col.names=F)

dist_lcss_diff_2=TSDatabaseDistances(diff_train_2,distance='lcss',epsilon=0.05)
dist_lcss_diff_2=as.matrix(dist_lcss_diff_2)
diag(dist_lcss_diff_2)=large_number
fwrite(dist_lcss_diff_2,sprintf('%s%s/%s_lcss_diff2_epsilon_005.csv', distance_path, second_dataset, second_dataset),col.names=F)  

dist_erp_diff_2=TSDatabaseDistances(diff_train_2,distance='erp',g=0.5)
dist_erp_diff_2=as.matrix(dist_erp_diff_2)
diag(dist_erp_diff_2)=large_number
fwrite(dist_erp_diff_2,sprintf('%s%s/%s_erp_diff2_gap_005.csv', distance_path, second_dataset, second_dataset),col.names=F)  

x being coerced from class: matrix to data.table
x being coerced from class: matrix to data.table
x being coerced from class: matrix to data.table
x being coerced from class: matrix to data.table


<a id="9"></a>
## PAA Datasets

### First PAA dataset when segment length value is 8

In [32]:
dist_euc_paa=as.matrix(dist(paa_train))
diag(dist_euc_paa)=large_number
fwrite(dist_euc_paa,sprintf('%s%s/%s_euc_paa_dist.csv', distance_path, second_dataset, second_dataset),col.names=F)

dist_dtw_paa=as.matrix(dtwDist(paa_train))
diag(dist_dtw_paa)=large_number
fwrite(dist_dtw_paa,sprintf('%s%s/%s_dtw_paa_dist.csv', distance_path, second_dataset, second_dataset),col.names=F)

dist_lcss_paa=TSDatabaseDistances(paa_train,distance='lcss',epsilon=0.05)
dist_lcss_paa=as.matrix(dist_lcss_paa)
diag(dist_lcss_paa)=large_number
fwrite(dist_lcss_paa,sprintf('%s%s/%s_lcss_paa_epsilon_005.csv', distance_path, second_dataset, second_dataset),col.names=F)  

dist_erp_paa=TSDatabaseDistances(paa_train,distance='erp',g=0.5)
dist_erp_paa=as.matrix(dist_erp_paa)
diag(dist_erp_paa)=large_number
fwrite(dist_erp_paa,sprintf('%s%s/%s_erp_paa_gap_005.csv', distance_path, second_dataset, second_dataset),col.names=F)  

x being coerced from class: matrix to data.table
x being coerced from class: matrix to data.table
x being coerced from class: matrix to data.table
x being coerced from class: matrix to data.table


### Second PAA dataset when segment length value is 16

In [33]:
dist_euc_paa_2=as.matrix(dist(paa_train_2))
diag(dist_euc_paa_2)=large_number
fwrite(dist_euc_paa_2,sprintf('%s%s/%s_euc_paa2_dist.csv', distance_path, second_dataset, second_dataset),col.names=F)

dist_dtw_paa_2=as.matrix(dtwDist(paa_train_2))
diag(dist_dtw_paa_2)=large_number
fwrite(dist_dtw_paa_2,sprintf('%s%s/%s_dtw_paa2_dist.csv', distance_path, second_dataset, second_dataset),col.names=F)

dist_lcss_paa_2=TSDatabaseDistances(paa_train_2,distance='lcss',epsilon=0.05)
dist_lcss_paa_2=as.matrix(dist_lcss_paa_2)
diag(dist_lcss_paa_2)=large_number
fwrite(dist_lcss_paa_2,sprintf('%s%s/%s_lcss_paa2_epsilon_005.csv', distance_path, second_dataset, second_dataset),col.names=F)  

dist_erp_paa_2=TSDatabaseDistances(paa_train_2,distance='erp',g=0.5)
dist_erp_paa_2=as.matrix(dist_erp_paa_2)
diag(dist_erp_paa_2)=large_number
fwrite(dist_erp_paa_2,sprintf('%s%s/%s_erp_paa2_gap_005.csv', distance_path, second_dataset, second_dataset),col.names=F)  

x being coerced from class: matrix to data.table
x being coerced from class: matrix to data.table
x being coerced from class: matrix to data.table
x being coerced from class: matrix to data.table


### To store obtained distance datasetes in a list

In [34]:
dist_folder=sprintf('%s%s', distance_path, second_dataset)
dist_files=list.files(dist_folder, full.names=T)

In [35]:
dist_files

[1] "C:/Users/bahad/GitHub/IE48B/Homework3/Distances/BirdChicken/BirdChicken_dtw_diff_dist.csv"         
 [2] "C:/Users/bahad/GitHub/IE48B/Homework3/Distances/BirdChicken/BirdChicken_dtw_diff2_dist.csv"        
 [3] "C:/Users/bahad/GitHub/IE48B/Homework3/Distances/BirdChicken/BirdChicken_dtw_paa_dist.csv"          
 [4] "C:/Users/bahad/GitHub/IE48B/Homework3/Distances/BirdChicken/BirdChicken_dtw_paa2_dist.csv"         
 [5] "C:/Users/bahad/GitHub/IE48B/Homework3/Distances/BirdChicken/BirdChicken_dtw_raw_dist.csv"          
 [6] "C:/Users/bahad/GitHub/IE48B/Homework3/Distances/BirdChicken/BirdChicken_erp_diff_gap_005.csv"      
 [7] "C:/Users/bahad/GitHub/IE48B/Homework3/Distances/BirdChicken/BirdChicken_erp_diff2_gap_005.csv"     
 [8] "C:/Users/bahad/GitHub/IE48B/Homework3/Distances/BirdChicken/BirdChicken_erp_paa_gap_005.csv"       
 [9] "C:/Users/bahad/GitHub/IE48B/Homework3/Distances/BirdChicken/BirdChicken_erp_paa2_gap_005.csv"      
[10] "C:/Users/bahad/GitHub/IE48B/Homework3/Distances/BirdChicken/BirdChicken_erp_raw_gap_005.csv"       
[11] "C:/Users/bahad/GitHub/IE48B/Homework3/Distances/BirdChicken/BirdChicken_euc_diff_dist.csv"         
[12] "C:/Users/bahad/GitHub/IE48B/Homework3/Distances/BirdChicken/BirdChicken_euc_diff2_dist.csv"        
[13] "C:/Users/bahad/GitHub/IE48B/Homework3/Distances/BirdChicken/BirdChicken_euc_paa_dist.csv"          
[14] "C:/Users/bahad/GitHub/IE48B/Homework3/Distances/BirdChicken/BirdChicken_euc_paa2_dist.csv"         
[15] "C:/Users/bahad/GitHub/IE48B/Homework3/Distances/BirdChicken/BirdChicken_euc_raw_dist.csv"          
[16] "C:/Users/bahad/GitHub/IE48B/Homework3/Distances/BirdChicken/BirdChicken_lcss_diff_epsilon_005.csv" 
[17] "C:/Users/bahad/GitHub/IE48B/Homework3/Distances/BirdChicken/BirdChicken_lcss_diff2_epsilon_005.csv"
[18] "C:/Users/bahad/GitHub/IE48B/Homework3/Distances/BirdChicken/BirdChicken_lcss_paa_epsilon_005.csv"  
[19] "C:/Users/bahad/GitHub/IE48B/Homework3/Distances/BirdChicken/BirdChicken_lcss_paa2_epsilon_005.csv" 
[20] "C:/Users/bahad/GitHub/IE48B/Homework3/Distances/BirdChicken/BirdChicken_lcss_raw_epsilon_005.csv"

<a id="10"></a>
# Main Model

In [36]:
k_levels=c(1,3,5)
approach_file=list.files(dist_folder)

In [37]:
result=vector('list',length(dist_files)*nof_rep*n_fold*length(k_levels))

In [38]:
iter=1
for(m in 1:length(dist_files)){ #
    print(dist_files[m])
    dist_mat=as.matrix(fread(dist_files[m],header=FALSE))
    for(i in 1:nof_rep){
        this_fold=cv_indices[[i]]
        for(j in 1:n_fold){
            test_indices=this_fold[[j]]
            for(k in 1:length(k_levels)){
                current_k=k_levels[k]
                current_fold=nn_classify_cv(dist_mat,trainclass,test_indices,k=current_k)
                accuracy=sum(trainclass[test_indices]==current_fold$prediction$predicted)/length(test_indices)
                tmp=data.table(approach=approach_file[m],repid=i,foldid=j,
                               k=current_k,acc=accuracy)
                result[[iter]]=tmp
                iter=iter+1
            }
            
        }
    
    }   
    
}

[1] "C:/Users/bahad/GitHub/IE48B/Homework3/Distances/BirdChicken/BirdChicken_dtw_diff_dist.csv"
[1] "C:/Users/bahad/GitHub/IE48B/Homework3/Distances/BirdChicken/BirdChicken_dtw_diff2_dist.csv"
[1] "C:/Users/bahad/GitHub/IE48B/Homework3/Distances/BirdChicken/BirdChicken_dtw_paa_dist.csv"
[1] "C:/Users/bahad/GitHub/IE48B/Homework3/Distances/BirdChicken/BirdChicken_dtw_paa2_dist.csv"
[1] "C:/Users/bahad/GitHub/IE48B/Homework3/Distances/BirdChicken/BirdChicken_dtw_raw_dist.csv"
[1] "C:/Users/bahad/GitHub/IE48B/Homework3/Distances/BirdChicken/BirdChicken_erp_diff_gap_005.csv"
[1] "C:/Users/bahad/GitHub/IE48B/Homework3/Distances/BirdChicken/BirdChicken_erp_diff2_gap_005.csv"
[1] "C:/Users/bahad/GitHub/IE48B/Homework3/Distances/BirdChicken/BirdChicken_erp_paa_gap_005.csv"
[1] "C:/Users/bahad/GitHub/IE48B/Homework3/Distances/BirdChicken/BirdChicken_erp_paa2_gap_005.csv"
[1] "C:/Users/bahad/GitHub/IE48B/Homework3/Distances/BirdChicken/BirdChicken_erp_raw_gap_005.csv"
[1] "C:/Users/bahad/GitHub/

<a id="11"></a>
# Result of Models

In [39]:
dataframe_result=rbindlist(result)
head(dataframe_result)

approach,repid,foldid,k,acc
BirdChicken_dtw_diff_dist.csv,1,1,1,1
BirdChicken_dtw_diff_dist.csv,1,1,3,1
BirdChicken_dtw_diff_dist.csv,1,1,5,1
BirdChicken_dtw_diff_dist.csv,1,2,1,1
BirdChicken_dtw_diff_dist.csv,1,2,3,1
BirdChicken_dtw_diff_dist.csv,1,2,5,1


In this dataset, result of each fold exists in this dataframe. repid and foldid represent which repetition and fold respectively. 

### Accumulated Datasets

In [40]:
acc_res=dataframe_result[,list(avg_acc=mean(acc),sdev_acc=sd(acc), repid=max(repid), foldid=max(foldid), 
                                   result_count=.N),by=list(approach,k)]
acc_res_ordered=acc_res[order(avg_acc,decreasing = TRUE)]

Accumulated dataset are ordered by avg_acc value. 

In [41]:
acc_res_ordered

approach,k,avg_acc,sdev_acc,repid,foldid,result_count
BirdChicken_dtw_diff_dist.csv,1,0.91,0.1940440,5,10,50
BirdChicken_dtw_diff2_dist.csv,1,0.90,0.2020305,5,10,50
BirdChicken_dtw_raw_dist.csv,1,0.85,0.2314550,5,10,50
BirdChicken_dtw_diff2_dist.csv,3,0.81,0.2451572,5,10,50
BirdChicken_erp_diff2_gap_005.csv,1,0.81,0.2451572,5,10,50
BirdChicken_dtw_diff_dist.csv,3,0.80,0.2474358,5,10,50
BirdChicken_dtw_paa2_dist.csv,1,0.77,0.2517287,5,10,50
BirdChicken_euc_paa_dist.csv,3,0.76,0.3070598,5,10,50
BirdChicken_euc_raw_dist.csv,3,0.76,0.3070598,5,10,50
BirdChicken_dtw_diff_dist.csv,5,0.75,0.2525381,5,10,50


In [42]:
# require(ggplot2)
# ggplot(dataframe_result,aes(x=paste0(approach,'with K=',k), y=acc)) +
#         geom_boxplot()+
#         labs(title="Boxplot of Models")+
#         xlab("Model Types")+
#         coord_flip()

## Best Result and Parameter Combination

In [55]:
acc_res_ordered[1]

approach,k,avg_acc,sdev_acc,repid,foldid,result_count
BirdChicken_dtw_diff_dist.csv,1,0.91,0.194044,5,10,50


Result is obtained by considering mean and standard deviation of the accuracy over the test folds.

### Best Parameter Combination

- Representation: 1 Difference Taken Data
- Distance: Dynamic Time Wrapping with epsilon 0.05
- K value of NN: 1

**Average Accuracy: 0.91 based on 5 repeats and 10-fold cross validation**

<a id="12"></a>
# Test Performance

Best performance is obtained when representation, distance, and k values are difference=1, Euclidean, and K=1 respectively.

In [43]:
traindata=as.matrix(fread(sprintf('%s%s/%s_TRAIN.txt',dataset_path, second_dataset,second_dataset)))
testdata=as.matrix(fread(sprintf('%s%s/%s_TEST.txt',dataset_path, second_dataset,second_dataset)))

To get test performance, train and test datasets will be used. These 2 datasets will be bind and test indices are selected as test dataset indices.

In [44]:
all_dt=rbind(traindata, testdata)

In [45]:
allclass=all_dt[,1] 
all_dt=all_dt[,2:ncol(all_dt)]

In [46]:
test_indices_last=(nrow(all_dt)+1-nrow(testdata)):nrow(all_dt)

### Parameters

In [47]:
test_indices_last

[1] 21 22 23 24 25 26 27 28 29 30 31 32 33 34 35 36 37 38 39 40

In [48]:
last_k=1

## Representation and Distance Calculation

Representation and distance types are selected by looking the best parameter combination obtained in train dataset.

In [49]:
diff_test=difference_obtainer(all_dt,1)

In [50]:
dist_dtw_diff_test=as.matrix(dtwDist(diff_test))
diag(dist_dtw_diff)=large_number

## Result of Test Dataset

In [51]:
last_result=nn_classify_cv(dist_dtw_diff_test,allclass,test_indices_last,k=last_k)
accuracy=sum(allclass[test_indices_last]==last_result$prediction$predicted)/length(test_indices_last)
final_res=data.table(approach="BirdChicken_dtw_diff_dist_Test.csv", k=last_k, acc=accuracy)

### Test Result

In [52]:
final_res

approach,k,acc
BirdChicken_dtw_diff_dist_Test.csv,1,0.75


### Train Result

In [53]:
acc_res_ordered[1][,c("approach", "k", "avg_acc")]

approach,k,avg_acc
BirdChicken_dtw_diff_dist.csv,1,0.91


<a id="13"></a>
# Comments

Obtained results will be analyzed in the result comparison notebook.